# Keras LSTM Text Generation
Text generation is a important nlp problem which can enable computers to write.

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [5]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
#from keras.optimizers import RMSprop
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
from google.colab import drive

In [6]:
!wget -O sherlock_homes.txt http://www.gutenberg.org/files/1661/1661-0.txt

--2022-03-28 09:06:05--  http://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/1661/1661-0.txt [following]
--2022-03-28 09:06:05--  https://www.gutenberg.org/files/1661/1661-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607430 (593K) [text/plain]
Saving to: ‘sherlock_homes.txt’

sherlock_homes.txt  100%[===================>] 593.19K   646KB/s    in 0.9s    

2022-03-28 09:06:07 (646 KB/s) - ‘sherlock_homes.txt’ saved [607430/607430]



In [7]:

drive.mount('/content/gdrive') 
#FILE=open('/content/gdrive/My Drive/H0.txt', 'r')               
#sample = os.path.join("", '/content/gdrive/My Drive/"alice.txt"')
sample = open("/content/gdrive/My Drive/Colab Notebooks/HP1.txt", "r") 
text = sample.read() 
print('text length', len(text))
#text = open('sherlock_homes.txt', 'r').read().lower()
#print('text length', len(text))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
text length 442744


In [8]:
print(text[:1000])

﻿Harry Potter
and the Philosopher?s Stone
J. K. Rowling

All rights reserved; no part of this publication may be reproduced or transmitted by any means, electronic, mechanical, photocopying or otherwise, without the prior permission of the publisher

First published in Great Britain in 1997
Bloomsbury Publishing Plc, 36 Soho Square, London, W1D 3QY

This edition first published in 2004

Copyright ? 1997 J. K. Rowling

Harry Potter, names, characters and related indicia are copyright and trademark Warner Bros., 2000?

The moral right of the author has been asserted
A CIP catalogue record of this book is available from the British Library

ISBN 978 0 7475 7360 9

The paper this book is printed on is certified by the ? 1996 Forest Stewardship Council A.C. (FSC). It is ancient-forest friendly. The printer holds FSC chain of custody SGS-COC-2061.

Printed in Great Britain by Clays Ltd, St Ives plc
Typeset by Dorchester Typesetting

5 7 9 10 8 6 4

www.bloomsbury.com/harrypotter

BLOOMSBURY


## Map chars to integers

In [9]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  76


In [10]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

## Split up into subsequences

In [11]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 147568


In [12]:
print(sentences[:3])
print(next_chars[:3])

['\ufeffHarry Potter\nand the Philosopher?s Ston', 'rry Potter\nand the Philosopher?s Stone\nJ', ' Potter\nand the Philosopher?s Stone\nJ. K']
['e', '.', '.']


In [13]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [14]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ...  True False False]
  ...
  [False False False ... False False False]
  [ True False False ... False False False]
  [False False False ... False False False]]

 [[False  True False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

## Building Model

In this notebook a small recurrent neural networks is used for both simplicity and because of the training time but if you want to train a more sophisticated model you can increase the size of the network. You can also use a model pretrained on some other text like wikipedia text to both speed up the training process and get better results.

In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [16]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Helper Functions
These helper functions are taken from the [official Keras text generation notebook](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).

In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Defining callbacks

In [19]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [20]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [21]:
callbacks = [print_callback, checkpoint, reduce_lr]

## Training the model

In [22]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
1152/1153 [============================>.] - ETA: 0s - loss: 1.9923
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "stopped to draw a deep breath and then w"
stopped to draw a deep breath and then was said and him the for the toll the room the rown and all the for the right and so the toll him. ? he said the room and the room and all the some and Harry with the right all him the toll the right him the hood and him the room and some so done stard and him the to look and so the room and the room and the right and him the room and so the for the first the round and the room and all the book and
----- diversity: 0.5
----- Generating with seed: "stopped to draw a deep breath and then w"
stopped to draw a deep breath and then was s to the had all him trough that work the rechoom to are that the on the Snape was never was the Harry that the room the that the door all as he for ? who could him the down that him the right didn?t had found of the 

## Testing the model
Now that we have a trained network we can test it using a method simular to the ``on_epoch_end`` method above.

In [23]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [24]:
print(generate_text(500, 0.2))

ck across the road, hurried up to his offing the stares and the straight the street to the troll was the troll was him.
  ?What?s the should here to the street to the boady and the street the stares and they were to the boady and the stand the hat had to have to the they were the train to the wall was a straight to the portrait was the corridor was the time the starent and still the stand and who was a large back and there was a straight to the stand on the floor of the stool spoke the stand back and sight as the stares and there was 
